In [4]:
import plotly.graph_objects as go
import math, random, time
from IPython.display import display
from IPython.display import clear_output


In [5]:
class ScanFrame:
    def __init__(self, theta_range, phi_range):
        self.theta_range = list(theta_range)
        self.phi_range = list(phi_range)
        self.grid = [[None for _ in self.phi_range] for _ in self.theta_range]

    def fill(self, t_idx, p_idx, r):
        self.grid[t_idx][p_idx] = r

    def is_full(self):
        return all(r is not None for row in self.grid for r in row)

    def to_cartesian(self):
        x, y, z = [], [], []
        for ti, theta in enumerate(self.theta_range):
            for pi, phi in enumerate(self.phi_range):
                r = self.grid[ti][pi]
                if r is not None:
                    theta_rad = math.radians(theta)
                    phi_rad = math.radians(phi)
                    # 這裡我們依你的 XZ 展開，Y 為距離邏輯
                    x.append(r * math.cos(theta_rad))
                    z.append(r * math.sin(phi_rad))
                    y.append(r)
        return x, y, z


In [6]:
# 初始資料
x_vals, y_vals, z_vals = [], [], []

# 建立空圖
fig = go.FigureWidget()
scatter = go.Scatter3d(
    x=[],
    y=[],
    z=[],
    mode='markers',
    marker=dict(size=4, color=[], colorscale='Viridis', opacity=0.8)
)
fig.add_trace(scatter)

fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0, y=2, z=0)  # 垂直從上往下看
    ),
    scene=dict(
        xaxis=dict(title='X (cm)', range=[-100, 100], dtick=20, autorange=False),
        yaxis=dict(title='Y (cm)', range=[0, 150], dtick=20, autorange=False),
        zaxis=dict(title='Z (cm)', range=[0, 100], dtick=20, autorange=False),
        bgcolor="black"
    ),
    title='3D 聲納掃描模擬（每秒刷新）'
)

display(fig)


FigureWidget({
    'data': [{'marker': {'color': [],
                         'colorscale': [[0.0, '#440154'], [0.1111111111111111,
                                        '#482878'], [0.2222222222222222,
                                        '#3e4989'], [0.3333333333333333,
                                        '#31688e'], [0.4444444444444444,
                                        '#26828e'], [0.5555555555555556,
                                        '#1f9e89'], [0.6666666666666666,
                                        '#35b779'], [0.7777777777777778,
                                        '#6ece58'], [0.8888888888888888,
                                        '#b5de2b'], [1.0, '#fde725']],
                         'opacity': 0.8,
                         'size': 4},
              'mode': 'markers',
              'type': 'scatter3d',
              'uid': '454cec41-0d03-4d7c-95a9-c76861de1e2c',
              'x': [],
              'y': [],
              'z': []}],
    'lay

In [7]:
theta_range = range(0, 180, 20)
phi_range = range(30, 90, 15)

# 迴圈模擬 20 頁掃描資料
for _ in range(20):
    scan = ScanFrame(theta_range, phi_range)  # 每輪一頁
    for ti, t in enumerate(theta_range):
        for pi, p in enumerate(phi_range):
            r = random.uniform(50, 100)
            scan.fill(ti, pi, r)

    # 將本輪掃描結果轉換為 x, y, z
    new_x, new_y, new_z = scan.to_cartesian()

    # 更新圖形
    with fig.batch_update():
        fig.data[0].x = new_x
        fig.data[0].y = new_y
        fig.data[0].z = new_z
        fig.data[0].marker.color = new_z

    time.sleep(0.5)
